<h1 style="text-align:center"><div style="text-style:bold; font-family:Verdana;font-size=100px">Analiza fantazijskih knjig</div></h1>

<h2 style='text-align:center'> <div style="font-size : 30px; font-style : Verdana; text-align : center "> Priprava </div> </h2>

In [1]:
import pandas as pd
import zlato_desetletje as zld

In [2]:
podatki = pd.read_csv("C:\\Users\\Benjamin\\Desktop\\Šola\\FMF\\UVP\\UVP_Projektna_naloga_2024\\fantazijska_literatura.csv", index_col='Naslov')

In [3]:
podatki.fillna(0, inplace=True)

In [4]:
# Sprememba decimalnih števil, ki bi morala biti cela, v cela števila.
podatki['Dolžina'] = podatki['Dolžina'].astype(int, errors='ignore')
podatki['Leta izida'] = podatki['Leta izida'].astype(int, errors='ignore')

In [16]:
podatki

In [6]:
podatki['Desetletje'] = (podatki['Leta izida']//10)*10

In [7]:
podatki = podatki[['Avtor', 'Leta izida', 'Desetletje', 'Dolžina', 'Točke', 'Ocena', 'Število bralcev']]

In [8]:
podatki

In [9]:
po_desetletjih = podatki.groupby('Desetletje')

<h2 style='text-align:center'> <div style="font-size : 30px; font-style : Verdana; text-align : center "> Lestvica najboljših </div> </h2>

<p style="font-family:'Times New Roman', font-size:12px"> Najprej si poglejmo najboljših 10 predstavnikov iz vsake kategorije. Z rezultati si bo lahko vsak ljubitelj fantazijskega žanra pomagal pri odločitvi, katere knjige oz. avtorje se najbolj splača brati.</p>

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Najkvalitetnejše knjige</h3>

<p style="font-family:'Times New Roman', font-size:12px"> Poizvedimo najprej, katere knjige so najkvalitetnejše. Kot merilo kvalitete vzemimo št. točk oz. t. i. "score", kakor je navedeno na spletni strani. Točke upoštevajo št. bralcev, ki je ocenilo knjigo, ter povprečno oceno, ki so ji jo prisodili.</p>

In [10]:
naj_knjige_po_tockah = pd.DataFrame(podatki['Točke'].sort_values(ascending=False))

In [11]:
naj_knjige_po_tockah.reset_index(inplace=True)

In [12]:
naj_knjige_po_tockah.index += 1

In [13]:
naj_knjige_po_tockah.head(10)

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Najkvalitetnejši avtorji</h3>

<p style="font-family:'Times New Roman', font-size:12px"> Sedaj analizirajmo avtorje po kvaliteti knjig, ki so jih napisali. Kot merilo zopet vzemimo število točk, le da tokrat avtorju pripišimo povprečje točk njegovih knjig.</p>

In [14]:
po_avtorjih = podatki.groupby('Avtor')

In [15]:
naj_avtorji_po_tockah = (pd.DataFrame(avtorji['Točke'].mean())).sort_values(by='Točke',ascending=False)

In [ ]:
naj_avtorji_po_tockah['Točke'] = round(avtorji_po_tockah['Točke'], 2)

In [ ]:
naj_avtorji_po_tockah.reset_index(inplace=True)

In [ ]:
naj_avtorji_po_tockah.index += 1

In [ ]:
naj_avtorji_po_tockah.head(10)

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Najbolj brane knjige</h3>

<p style="font-family:'Times New Roman', font-size:12px"> Kot zanimivost si poglejmo še, katere so najbolj brane knjige in najbolj brani avtorji ter rezultata primerjajmo z zgornjima. Kot kriterij sem vzel število ljudi, ki je ocenilo knjigo na spletni strani, saj dejanskega števila ljudi, ki so prebrali knjigo, ni mogoče zares ugotoviti.</p>

In [ ]:
naj_knjige_po_bralcih = podatki.sort_values(by='Število bralcev', ascending=False)

In [ ]:
naj_knjige_po_bralcih = pd.DataFrame(najbolj_brane_knjige['Število bralcev'])

In [ ]:
naj_knjige_po_bralcih.reset_index(inplace=True)

In [ ]:
naj_knjige_po_bralcih.index += 1

In [ ]:
naj_knjige_po_bralcih.head(10)

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Najbolj brani avtorji</h3>

<p style="font-family:'Times New Roman', font-size:12px"> Še po avtorjih </p>

In [ ]:
naj_avtorji_po_bralcih = (pd.DataFrame(avtorji['Število bralcev'].mean())).sort_values(by='Število bralcev', ascending=False)

In [ ]:
naj_avtorji_po_bralcih.reset_index(inplace=True)

In [ ]:
naj_avtorji_po_bralcih.index += 1

In [ ]:
naj_avtorji_po_bralcih.head(10)

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Sklepi</h3>

<h2 style='text-align:center'> <div style="font-size : 30px; font-style : Verdana; text-align : center "> Analiza trendov </div> </h2>

<p style="font-family:'Times New Roman', font-size:12px"> Zanima nas tudi podrobnejše dogajanje v fantazijskem žanru.<br> Kot bralci bi se lahko vprašali, kako se tekom let in desetletij spreminjajo kvaliteta, dolžina in branost književnih del. Prav tako bi vsekakor želeli vedeti, ali so daljše knjige tudi kvalitetnejše ter ali ljudje raje berejo daljše ali krajše knjige.<br> Iz rezultatov analize bomo poskusili napovedati trende na tem področju v prihodnosti.</p>

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Odvisnost kvalitete knjige od leta izida</h3>

In [ ]:
izid_leto = po_desetletjih.agg(
**{
    'Kvaliteta': pd.NamedAgg(column='Točke', aggfunc='mean')
})

In [ ]:
izid_leto

In [ ]:
izid_leto.drop(0, axis='index', inplace=True)  # Knjig brez navedenega leta izida ne upoštevamo.

In [ ]:
izid_leto['Kvaliteta'] = round(izid_leto['Kvaliteta'], 2)

In [ ]:
izid_leto.plot(ylabel='Točke')

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Odvisnost dolžine knjige od leta izida</h3>

In [ ]:
leto_dolzina = po_desetletjih.agg(
    **{
'Dolžina': pd.NamedAgg(column='Dolžina', aggfunc='mean')
    })

In [ ]:
leto_dolzina.drop(0, inplace=True)

In [ ]:
leto_dolzina.plot(ylabel='Povprečna dolžina knjig (strani)')

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Odvisnost kvalitete knjige od njene dolžine</div></h3>

<p style="font-family:'Times New Roman', font-size:12px"> Radi bi videli, ali so daljše knjige kvalitetnejše. Pogledali bomo, koliko točk povprečno prejme vsaka knjiga iz posameznega razreda, pri čemer en razred predstavljajo vse tiste knjige, katerih števila strani dajo pri deljenju s 50 isti celi del.</p>

In [ ]:
dolzina_kvaliteta = podatki[['Dolžina', 'Točke']]

In [ ]:
dolzina_kvaliteta['Razred dolžine'] = (dolzina_kvaliteta['Dolžina']//50)*50

In [ ]:
dolzina_kvaliteta

In [ ]:
po_dolzini = dolzina_kvaliteta.groupby('Razred dolžine')

In [ ]:
dolzina_tocke = pd.DataFrame(po_dolzini['Točke'].mean())

In [ ]:
dolzina_tocke

In [ ]:
dolzina_tocke.plot(xlabel='Razred dolžine', ylabel='Točke')

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Odvisnost števila bralcev knjige od njene dolžine</div></h3>

In [ ]:
podatki['Razred dolžine'] = (podatki['Dolžina']//50)*50 

In [ ]:
po_razredih_dolzine = novi_podatki.groupby('Razred dolžine')

In [ ]:
dolzina_bralci = pd.DataFrame(po_razredih_dolzine['Število bralcev'].sum()) 

In [ ]:
dolzina_bralci.plot(ylabel='Skupno število bralcev (v milijonih)')

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Sklepi</div></h3>

<h2 style='text-align:center'> <div style="font-size : 30px; font-style : Verdana; text-align : center "> Zlata leta fantazijskega žanra </div> </h2>

<p style="font-family:'Times New Roman', font-size:12px"> Za konec si poglejmo še, katera so bila tista "zlata leta" fantazijskega žanra, tj. v katerem desetletju so nastajali najboljši fantazijski romani. <br> To bomo ugotovili tako, da bomo pogledali razvrstitev desetletij po določenih kategorijah, jim vsakič pripisali neko število točk, na koncu pa točke sešteli in kot najboljše desetletje razglasili tisto desetletje, ki bo imelo največ točk.</p>
<p style="font-family:'Times New Roman', font-size:12px"> Omenjene kategorije bodo: <ul>
    <li> število knjig, ki so bile napisane v desetletju,</li>
    <li> povprečna kvaliteta knjig,</li>
    <li> število bralcev, ki je prebralo knjige iz določenega desetletja in</li>
    <li> število avtorjev, ki je ustvarjalo v posameznem desetletju.</li>
    </ul></p>

In [ ]:
analiza_po_desetletjih = po_desetletjih.agg(
**{
    'Število knjig' : pd.NamedAgg(column='Točke', aggfunc='size'),
    'Kvaliteta' : pd.NamedAgg(column='Točke', aggfunc='mean'),
    'Število bralcev' : pd.NamedAgg(column='Število bralcev', aggfunc='sum'),
    'Število avtorjev' : pd.NamedAgg(column='Avtor', aggfunc=lambda x: len(x.unique()))
    
})

In [ ]:
analiza_po_desetletjih.drop(0, inplace=True)

In [ ]:
analiza_po_desetletjih['Kvaliteta'] = round(analiza_po_desetletjih['Kvaliteta'], 2)

In [ ]:
analiza_po_desetletjih

In [ ]:
# Beleženje rezultatov po kategorijah
zlate_tocke = {
    'Desetletje': [1900 + i*10 for i in range(2,13)],
    'Delež':[],
    'Kvaliteta':[],
    'Število bralcev':[],
    'Število avtorjev':[],
}

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Delež knjig, ki so izšle v posameznih desetletjih</div></h3>

In [ ]:
analiza_st_knjig = pd.DataFrame(analiza_po_desetletjih['Število knjig'])

In [ ]:
analiza_st_knjig.reset_index(inplace=True)

In [ ]:
analiza_st_knjig.index += 1

In [ ]:
analiza_st_knjig

In [ ]:
analiza_po_desetletjih['Število knjig'].plot(kind='pie', figsize=(20, 20), fontsize=20, ylabel=None)

In [ ]:
zlate_tocke['Delež'] = zld.mesto_v_tabeli(analiza_st_knjig, 'Število knjig')

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Kvaliteta knjig</div></h3>

In [ ]:
analiza_kvalitete = pd.DataFrame(analiza_po_desetletjih['Kvaliteta'])

In [ ]:
analiza_po_desetletjih['Kvaliteta'].plot(kind='bar', ylabel='Povprečno število točk')

In [ ]:
zlate_tocke['Kvaliteta'] = zld.mesto_v_tabeli(analiza_kvalitete, 'Kvaliteta')

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Število bralcev</div></h3>

In [ ]:
analiza_po_desetletjih['Število bralcev'].plot(kind='bar', ylabel='Število bralcev')

In [ ]:
analiza_bralci = pd.DataFrame(analiza_po_desetletjih['Število bralcev'])

In [ ]:
analiza_bralci.plot(kind='bar' ylabel='Število bralcev')

In [ ]:
zlate_tocke['Število bralcev'] = zld.mesto_v_tabeli(analiza_bralci, 'Število bralcev')

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Število piscev fantazijskih romanov</div></h3>

In [ ]:
analiza_po_desetletjih['Število avtorjev'].plot(kind='bar', ylabel='Število ustvarjalcev v obdobju')

In [ ]:
analiza_avtorji = pd.DataFrame(analiza_po_desetletjih['Število avtorjev'])

In [ ]:
analiza_avtorji.plot(kind='bar', ylabel='Število avtorjev')

In [ ]:
zlate_tocke['Število avtorjev'] = zld.mesto_v_tabeli(st_avtorjev, 'Število avtorjev')

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Iskanje zlatega desetletja</div></h3>

In [ ]:
vrstni_red = pd.DataFrame(zlate_tocke)

In [ ]:
vrstni_red

In [ ]:
# Računanje vsot prejetih točk
seznam_vsot = []
for i in range(len(vrstni_red)):
    vsota = 0
    for j in range(1, 5):
        vsota += vrstni_red.iloc[i, j]
    seznam_vsot.append(vsota)

slovar = {'Desetletje':[1920 + i*10 for i in range(0, 11)], 'Skupne točke' : seznam_vsot}
skupne_tocke = pd.DataFrame(slovar)
skupne_tocke

In [ ]:
vrstni_red_po_tockah = vrstni_red.merge(skupne_tocke, how='outer', on='Desetletje')

In [ ]:
vrstni_red_po_tockah.sort_values(by=['Skupne točke', 'Desetletje'], ascending=[False, True], inplace=True)

In [ ]:
vrstni_red_po_tockah.reset_index(inplace=True)

In [ ]:
vrstni_red_po_tockah.index += 1

In [ ]:
vrstni_red_po_tockah.drop(columns=["index"], inplace=True)

In [ ]:
vrstni_red_po_tockah

In [ ]:
graf_desetletij = vrstni_red_po_tockah[['Desetletje', 'Delež', 'Kvaliteta', 'Število bralcev', 'Število avtorjev']].set_index('Desetletje').plot(kind='bar', stacked = True)

<h3 style="text-align:center"><div style="font-size : 20px; font-style : Verdana; text-align : center">Sklepi</div></h3>

<h2 style='text-align:center'> <div style="font-size : 30px; font-style : Verdana; text-align : center "> Zaključki </div> </h2>

<p style="font-family:'Times New Roman', font-size:12px">Iz rezultatov analize, prikazanih zgoraj, sledi</p> 